In [7]:
import os
import json
import numpy as np
import faiss
from openai import OpenAI

# OpenAI 초기화
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_embedding(text):
    try:
        response = openai_client.embeddings.create(input=[text], model="text-embedding-ada-002")
        return response.data[0].embedding
    except Exception as e:
        print(f"임베딩 생성 중 오류 발생: {str(e)}")
        return None

def store_job_data(jobs):
    total_jobs = len(jobs)
    successful_uploads = 0
    embeddings = []
    job_metadata = []
    
    for i, job in enumerate(jobs, 1):
        try:
            job_info = f"""
            메인 카테고리: {job['main_category']}
            직업군: {job['job_class']}
            직업: {job['name']}
            주스탯: {job['stat']}
            """
            embedding = get_embedding(job_info)
            if embedding is None:
                print(f"직업 {i}/{total_jobs} 처리 실패: {job['name']} - 임베딩 생성 실패")
                continue
            
            embeddings.append(embedding)
            job_metadata.append({
                "main_category": job['main_category'],
                "job_class": job['job_class'],
                "job_name": job['name'],
                "stat": job['stat']
            })
            successful_uploads += 1
            print(f"직업 {i}/{total_jobs} 처리 완료: {job['name']}")
        except Exception as e:
            print(f"직업 {i}/{total_jobs} 처리 실패: {job['name']} - 오류: {str(e)}")
    
    print(f"\n총 {total_jobs}개 중 {successful_uploads}개 직업 데이터 처리 완료")
    return np.array(embeddings).astype('float32'), job_metadata

# JSON 파일 읽기
with open('maplestory_jobs.json', 'r', encoding='utf-8') as file:
    jobs = json.load(file)

print(f"JSON 파일에서 {len(jobs)}개의 직업 데이터를 로드했습니다.")

# 임베딩 생성 및 FAISS 인덱스 생성
embeddings, job_metadata = store_job_data(jobs)

dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# FAISS 인덱스 저장
faiss.write_index(index, "maplestory_jobs_index.faiss")

print("FAISS 인덱스가 성공적으로 생성되고 저장되었습니다.")

# 메타데이터 저장
with open('maplestory_jobs_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(job_metadata, f, ensure_ascii=False, indent=4)

print("직업 메타데이터가 성공적으로 저장되었습니다.")

# 검색 함수
def search_jobs(query, k=5):
    query_embedding = get_embedding(query)
    D, I = index.search(np.array([query_embedding]).astype('float32'), k)
    results = []
    for i, idx in enumerate(I[0]):
        results.append((job_metadata[idx], D[0][i]))
    return results

# 검색 예제
query = "힘을 주스탯으로 사용하는 전사 직업"
results = search_jobs(query)
print(f"\n검색 쿼리: {query}")
for job, distance in results:
    print(f"직업: {job['job_name']} ({job['job_class']})")
    print(f"주스탯: {job['stat']}")
    print(f"유사도 거리: {distance}")
    print()

JSON 파일에서 46개의 직업 데이터를 로드했습니다.
직업 1/46 처리 완료: 히어로
직업 2/46 처리 완료: 아크메이지(불,독)
직업 3/46 처리 완료: 팔라딘
직업 4/46 처리 완료: 다크나이트
직업 5/46 처리 완료: 아크메이지(썬,콜)
직업 6/46 처리 완료: 비숍
직업 7/46 처리 완료: 신궁
직업 8/46 처리 완료: 보우마스터
직업 9/46 처리 완료: 패스파인더
직업 10/46 처리 완료: 듀얼블레이드
직업 11/46 처리 완료: 나이트로드
직업 12/46 처리 완료: 섀도어
직업 13/46 처리 완료: 캐논슈터
직업 14/46 처리 완료: 캡틴
직업 15/46 처리 완료: 바이퍼
직업 16/46 처리 완료: 소울마스터
직업 17/46 처리 완료: 미하일
직업 18/46 처리 완료: 플레임위자드
직업 19/46 처리 완료: 나이트워커
직업 20/46 처리 완료: 스트라이커
직업 21/46 처리 완료: 윈드브레이커
직업 22/46 처리 완료: 배틀메이지
직업 23/46 처리 완료: 블래스터
직업 24/46 처리 완료: 메카닉
직업 25/46 처리 완료: 와일드헌터
직업 26/46 처리 완료: 제논
직업 27/46 처리 완료: 데몬슬레이어
직업 28/46 처리 완료: 데몬어벤져
직업 29/46 처리 완료: 아란
직업 30/46 처리 완료: 에반
직업 31/46 처리 완료: 루미너스
직업 32/46 처리 완료: 팬텀
직업 33/46 처리 완료: 은월
직업 34/46 처리 완료: 메르세데스
직업 35/46 처리 완료: 카이저
직업 36/46 처리 완료: 엔젤릭버스터
직업 37/46 처리 완료: 카인
직업 38/46 처리 완료: 카데나
직업 39/46 처리 완료: 아델
직업 40/46 처리 완료: 일리움
직업 41/46 처리 완료: 아크
직업 42/46 처리 완료: 칼리
직업 43/46 처리 완료: 라라
직업 44/46 처리 완료: 호영
직업 45/46 처리 완료: 제로
직업 46/46 처리 완료: 키네시스

총 46개 중 46개 직업 데이

In [6]:
import json
import numpy as np
import faiss
from openai import OpenAI
import os

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# JSON 파일에서 보스 몬스터 데이터 로드
with open('maplestory_boss_monsters.json', 'r', encoding='utf-8') as f:
    boss_data = json.load(f)

# 임베딩 생성 함수
def get_embedding(text):
    response = client.embeddings.create(input=[text], model="text-embedding-ada-002")
    return response.data[0].embedding

# 보스 몬스터 데이터를 텍스트로 변환
boss_texts = []
for boss in boss_data['boss_monsters']:
    for difficulty in boss['difficulties']:
        text = f"보스: {boss['name']}, 난이도: {difficulty['difficulty']}, 레벨: {difficulty['level']}, HP: {difficulty['hp']}, 방어율: {difficulty['defense_rate']}, 포스: {difficulty['force']['type']} {difficulty['force']['value']}"
        boss_texts.append(text)

# 임베딩 생성
embeddings = [get_embedding(text) for text in boss_texts]

# FAISS 인덱스 생성 및 데이터 추가
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype('float32'))

# 인덱스 저장
faiss.write_index(index, "maplestory_boss_index.faiss")

print("FAISS 인덱스가 성공적으로 생성되고 저장되었습니다.")

# 검색 예제
def search_boss(query, k=5):
    query_embedding = get_embedding(query)
    D, I = index.search(np.array([query_embedding]).astype('float32'), k)
    return [(boss_texts[i], D[0][j]) for j, i in enumerate(I[0])]

# 검색 예제 실행
results = search_boss("레벨 200 이상의 강력한 보스")
for result, distance in results:
    print(f"결과: {result}")
    print(f"거리: {distance}")
    print()

FAISS 인덱스가 성공적으로 생성되고 저장되었습니다.
결과: 보스: 검은 마법사, 난이도: 하드, 레벨: 275, HP: 472500000000000, 방어율: 300, 포스: 아케인포스 1320
거리: 0.2486695796251297

결과: 보스: 반반, 난이도: 카오스, 레벨: 190, HP: 100000000000, 방어율: 120, 포스: None 0
거리: 0.25010767579078674

결과: 보스: 카링, 난이도: 하드, 레벨: 285, HP: 19200000000000000, 방어율: 380, 포스: 어센틱포스 350
거리: 0.25216448307037354

결과: 보스: 감시자 칼로스, 난이도: 이지, 레벨: 270, HP: 357000000000000, 방어율: 380, 포스: 어센틱포스 200
거리: 0.2548547387123108

결과: 보스: 림보, 난이도: 하드, 레벨: 285, HP: 14010000000000000, 방어율: 380, 포스: 어센틱포스 500
거리: 0.255917489528656



In [2]:
import json

# JSON 파일에서 무기 데이터 로드
with open('categorized_weapons.json', 'r', encoding='utf-8') as f:
    weapon_data = json.load(f)

# "메이플 1500일 깃발"을 제외하고 공백이나 하이픈을 0으로 변경
for category, weapons in weapon_data.items():
    weapon_data[category] = []
    for weapon in weapons:
        # 무기 이름이 "메이플 1500일 깃발", "메이플 깃발", "메이플 우산", "메이플 레전드 깃발" 중 하나가 아닐 경우만 추가
        if weapon.get("name") not in ["메이플 1500일 깃발", "메이플 깃발", "메이플 우산", "메이플 레전드 깃발",'메이플 1000일 깃발','레전드 메이플 깃발']:
            # 공백 또는 하이픈을 0으로 변경
            for key in ['attack_power', 'magic_attack', 'req_int', 'req_str', 'req_dex', 'req_luk', 'str', 'dex', 'luk', 'damage', 'boss_damage_ignore_defense', 'upgrade_slots']:
                if key in weapon:
                    if weapon[key] == '' or weapon[key] == '-':
                        weapon[key] = 0  # 공백 또는 하이픈이면 0으로 설정
                    else:
                        # 숫자로 변환 가능한 경우에만 정수형으로 변환
                        try:
                            weapon[key] = int(weapon[key])  # 정수형으로 변환 (문자열에서)
                        except ValueError:
                            weapon[key] = 0  # 변환할 수 없는 경우 기본값 설정

            # 무기 이름에 따라 보스 데미지 및 방어력 무시 값 변경
            weapon_name = weapon.get("name")
            if '파프니르' in weapon_name or '앱솔랩스' in weapon_name:
                weapon['boss_damage_ignore_defense'] = '30%/10%'
            elif '아케인셰이드' in weapon_name or '제네시스' in weapon_name:
                weapon['boss_damage_ignore_defense'] = '30%/20%'

            weapon_data[category].append(weapon)

# 수정된 데이터를 JSON 파일에 다시 저장
with open('categorized_weapons.json', 'w', encoding='utf-8') as f:
    json.dump(weapon_data, f, ensure_ascii=False, indent=2)

print('"메이플 깃발"들이 제거되고 공백 및 하이픈 값이 0으로 변경된 JSON 파일이 성공적으로 저장되었습니다.')


"메이플 깃발"들이 제거되고 공백 및 하이픈 값이 0으로 변경된 JSON 파일이 성공적으로 저장되었습니다.


In [3]:
import json
import numpy as np
import faiss
from openai import OpenAI
import os

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# JSON 파일에서 무기 데이터 로드
with open('categorized_weapons.json', 'r', encoding='utf-8') as f:
    weapon_data = json.load(f)

# 임베딩 생성 함수
def get_embedding(text):
    response = client.embeddings.create(input=[text], model="text-embedding-ada-002")
    return response.data[0].embedding

# 무기 데이터를 텍스트로 변환 (고유 정보를 추가하여 차별화)
weapon_texts = []
weapon_metadata = []
for category, weapons in weapon_data.items():
    for weapon in weapons:
        text = (
            f"카테고리: {category}, 이름: {weapon['name']}, "
            f"레벨: {weapon['req_level']}, 공격력: {weapon.get('attack_power', '없음')}, "
            f"마력: {weapon.get('magic_attack', '없음')}, STR: {weapon.get('str', '없음')}, "
            f"DEX: {weapon.get('dex', '없음')}, INT: {weapon.get('int', '없음')}, "
            f"LUK: {weapon.get('luk', '없음')}, 보스 방어율 무시: {weapon.get('boss_damage_ignore_defense', '없음')}, "
            f"업그레이드 가능 횟수: {weapon.get('upgrade_slots', '없음')}, 비고: {weapon.get('notes', '없음')}, "
            f"고유 아이템: {'[고유]' if '고유' in weapon.get('notes', '') else ''}"
        )
        weapon_texts.append(text)
        weapon_metadata.append({
            "category": category,
            "name": weapon["name"],
            "level": weapon["req_level"],
        })

# 중복 제거를 위한 집합 생성 (필요할 경우)
unique_texts = list(set(weapon_texts))

# 임베딩 생성
embeddings = [get_embedding(text) for text in unique_texts]

# FAISS 인덱스 생성 (CPU 버전)
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)  # L2 거리 기반 인덱스 생성

# 데이터 추가
index.add(np.array(embeddings).astype('float32'))

# 인덱스와 메타데이터 저장
faiss.write_index(index, "maplestory_weapon_index.faiss")
with open("weapon_metadata.json", 'w', encoding='utf-8') as f:
    json.dump(weapon_metadata, f, ensure_ascii=False, indent=2)

print("FAISS CPU 인덱스가 성공적으로 생성되고 저장되었습니다.")

# 검색 예제
def search_weapon(query, k=5):
    query_embedding = get_embedding(query)
    D, I = index.search(np.array([query_embedding]).astype('float32'), k)
    return [(unique_texts[i], D[0][j]) for j, i in enumerate(I[0])]

# 검색 예제 실행
results = search_weapon("레벨 150 이상의 강력한 마력 무기")
for result, distance in results:
    print(f"결과: {result}")
    print(f"거리: {distance}")
    print()

FAISS CPU 인덱스가 성공적으로 생성되고 저장되었습니다.
결과: 카테고리: 두손검, 이름: 해방된 [[카이세리움]], 레벨: 150, 공격력: 400, 마력: 0, STR: 200, DEX: 200, INT: , LUK: 0, 보스 방어율 무시: 0, 업그레이드 가능 횟수: 1, 비고: [*교불 교환 불가], 고유 아이템: 
거리: 0.27900728583335876

결과: 카테고리: 두손검, 이름: 레전드 참화도, 레벨: 100, 공격력: 105, 마력: 0, STR: 0, DEX: 0, INT: , LUK: 0, 보스 방어율 무시: 0, 업그레이드 가능 횟수: 7, 비고: [*장교불 장착시 교환 불가], 고유 아이템: 
거리: 0.2838771939277649

결과: 카테고리: 매직 건틀렛, 이름: 마티스 매직 건틀렛, 레벨: 30, 공격력: 27, 마력: 43, STR: 0, DEX: 0, INT: , LUK: 0, 보스 방어율 무시: 0, 업그레이드 가능 횟수: 7, 비고: [* 일리움이 일러스트에서 착용하고 있는 무기다.], 고유 아이템: 
거리: 0.2844869792461395

결과: 카테고리: 단검, 이름: 레전드 용천권, 레벨: 80, 공격력: 89, 마력: 0, STR: 0, DEX: 0, INT: , LUK: 0, 보스 방어율 무시: 0, 업그레이드 가능 횟수: 7, 비고: [*장교불], 고유 아이템: 
거리: 0.28658390045166016

결과: 카테고리: 두손둔기, 이름: 레전드 드래곤 플레임, 레벨: 110, 공격력: 112, 마력: 0, STR: 0, DEX: 0, INT: , LUK: 0, 보스 방어율 무시: 0, 업그레이드 가능 횟수: 7, 비고: [*장교불 장착시 교환 불가], 고유 아이템: 
거리: 0.28772616386413574



In [8]:
import asyncio
import aiohttp
import json
import os
from datetime import datetime, timedelta

import nest_asyncio
nest_asyncio.apply()

BASE_URL = "https://open.api.nexon.com/maplestory/v1"
NEXON_API = os.getenv("NEXON_API_KEY")

async def get_api_data(session, endpoint, params=None):
    headers = {"x-nxopen-api-key": NEXON_API}
    url = f"{BASE_URL}{endpoint}"
    try:
        async with session.get(url, headers=headers, params=params) as response:
            if response.status == 200:
                return await response.json()
            else:
                error_text = await response.text()
                print(f"API 호출 실패: {response.status} - {error_text}")
                print(f"요청 URL: {url}")
                print(f"요청 파라미터: {params}")
                return None
    except Exception as e:
        print(f"API 호출 중 오류 발생: {e}")
        return None

async def get_union_artifact_info(session, character_name):
    id_data = await get_api_data(session, "/id", {"character_name": character_name})
    if not id_data or 'ocid' not in id_data:
        print(f"{character_name}의 OCID를 가져오는데 실패했습니다.")
        return None

    ocid = id_data['ocid']
    yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
    params = {"ocid": ocid, "date": yesterday}
    
    print(f"유니온 아티팩트 정보 요청 파라미터: {params}")  # 디버그 출력
    
    union_artifact_info = await get_api_data(session, "/user/union-artifact", params)

    if union_artifact_info:
        return {character_name: union_artifact_info}
    else:
        print(f"{character_name}의 유니온 아티팩트 정보를 가져오는데 실패했습니다.")
        return None

async def get_multiple_characters_union_artifact_info(character_names):
    async with aiohttp.ClientSession() as session:
        tasks = [get_union_artifact_info(session, name) for name in character_names]
        results = await asyncio.gather(*tasks)
        return {list(result.keys())[0]: list(result.values())[0] for result in results if result}

def save_union_artifact_info_to_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def format_union_artifact_data(data):
    formatted_data = {
        "union_artifact_effect": [],
        "union_artifact_crystal": [],
        "union_artifact_remain_ap": data.get("union_artifact_remain_ap", 0)
    }
    
    for effect in data.get("union_artifact_effect", []):
        formatted_data["union_artifact_effect"].append({
            "name": effect.get("name", ""),
            "level": effect.get("level", 0)
        })
    
    for crystal in data.get("union_artifact_crystal", []):
        formatted_data["union_artifact_crystal"].append({
            "name": crystal.get("name", ""),
            "level": crystal.get("level", 0),
            "crystal_option_name_1": crystal.get("crystal_option_name_1", ""),
            "crystal_option_name_2": crystal.get("crystal_option_name_2", ""),
            "crystal_option_name_3": crystal.get("crystal_option_name_3", "")
        })
    
    return formatted_data

async def main():
    character_names = ["아델", "아크", "시프", "도적"]

    union_artifact_data = await get_multiple_characters_union_artifact_info(character_names)
    
    if union_artifact_data:
        for character_name, data in union_artifact_data.items():
            formatted_data = format_union_artifact_data(data)
            filename = f"maplestory_union_artifact_info_{character_name}.json"
            save_union_artifact_info_to_json({character_name: formatted_data}, filename)
            print(f"{character_name}의 유니온 아티팩트 정보가 {filename}에 저장되었습니다.")
    else:
        print("유니온 아티팩트 정보를 가져오는데 실패했습니다.")

if __name__ == "__main__":
    asyncio.run(main())

유니온 아티팩트 정보 요청 파라미터: {'ocid': '30ae5be8a9f2c9119f6cad798fffc391', 'date': '2024-10-29'}
유니온 아티팩트 정보 요청 파라미터: {'ocid': 'e0a4f439e53c369866b55297d2f5f4eb', 'date': '2024-10-29'}
유니온 아티팩트 정보 요청 파라미터: {'ocid': '840f4ad7377e5e5bf224bbcc975cceb0', 'date': '2024-10-29'}
유니온 아티팩트 정보 요청 파라미터: {'ocid': '1a9383057d237720b79136a802ff9f46', 'date': '2024-10-29'}
아델의 유니온 아티팩트 정보가 maplestory_union_artifact_info_아델.json에 저장되었습니다.
아크의 유니온 아티팩트 정보가 maplestory_union_artifact_info_아크.json에 저장되었습니다.
시프의 유니온 아티팩트 정보가 maplestory_union_artifact_info_시프.json에 저장되었습니다.
도적의 유니온 아티팩트 정보가 maplestory_union_artifact_info_도적.json에 저장되었습니다.


In [11]:
import json

def merge_json_files(file_names, output_file):
    merged_data = {
        "union_artifact_effect": [],
        "union_artifact_crystal": [],
        "union_artifact_remain_ap": 0
    }

    for file_name in file_names:
        with open(file_name, 'r', encoding='utf-8') as file:
            data = json.load(file)
            
            # data가 이미 원하는 구조라고 가정
            if "union_artifact_effect" in data:
                merged_data["union_artifact_effect"] = data["union_artifact_effect"]
            if "union_artifact_crystal" in data:
                merged_data["union_artifact_crystal"] = data["union_artifact_crystal"]
            if "union_artifact_remain_ap" in data:
                merged_data["union_artifact_remain_ap"] = max(
                    merged_data["union_artifact_remain_ap"],
                    data["union_artifact_remain_ap"]
                )

    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump(merged_data, outfile, ensure_ascii=False, indent=2)

# 합칠 JSON 파일들의 이름 리스트
file_names = [
    "maplestory_union_artifact_info_아델.json",
    "maplestory_union_artifact_info_아크.json",
    "maplestory_union_artifact_info_시프.json",
    "maplestory_union_artifact_info_도적.json"
]

# 결과 파일 이름
output_file = "merged_union_artifact_info.json"

# 파일 합치기 실행
merge_json_files(file_names, output_file)

print(f"모든 캐릭터의 유니온 아티팩트 정보가 {output_file}에 저장되었습니다.")

모든 캐릭터의 유니온 아티팩트 정보가 merged_union_artifact_info.json에 저장되었습니다.


In [ ]:
import asyncio
import aiohttp
import json
import os
from datetime import datetime, timedelta

import nest_asyncio
nest_asyncio.apply()

BASE_URL = "https://open.api.nexon.com/maplestory/v1"
NEXON_API = os.getenv("NEXON_API_KEY")

async def get_api_data(session, endpoint, params=None):
    headers = {"x-nxopen-api-key": NEXON_API}
    url = f"{BASE_URL}{endpoint}"
    try:
        async with session.get(url, headers=headers, params=params) as response:
            if response.status == 200:
                return await response.json()
            else:
                error_text = await response.text()
                print(f"API 호출 실패: {response.status} - {error_text}")
                return None
    except Exception as e:
        print(f"API 호출 중 오류 발생: {e}")
        return None

async def get_union_raider_info(session, character_name):
    id_data = await get_api_data(session, "/id", {"character_name": character_name})
    if not id_data or 'ocid' not in id_data:
        print(f"랭커의 OCID를 가져오는데 실패했습니다.")
        return None

    ocid = id_data['ocid']
    yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
    params = {"ocid": ocid, "date": yesterday}
    
    union_raider_info = await get_api_data(session, "/user/union-raider", params)

    if union_raider_info:
        # 날짜 정보 제거
        union_raider_info.pop('date', None)
        return union_raider_info
    else:
        print(f"랭커의 유니온 레이더 정보를 가져오는데 실패했습니다.")
        return None

async def get_multiple_characters_union_raider_info(character_names):
    async with aiohttp.ClientSession() as session:
        tasks = [get_union_raider_info(session, name) for name in character_names]
        results = await asyncio.gather(*tasks)
        return [result for result in results if result]

def save_union_raider_info_to_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump({"rankers": data}, f, ensure_ascii=False, indent=2)

async def main():
    character_names = ["아델", "아크", "시프", "도적", "맑음",'건물주','페이커','오지환']  # 원하는 캐릭터 이름 리스트

    union_raider_data = await get_multiple_characters_union_raider_info(character_names)
    
    if union_raider_data:
        filename = "maplestory_union_raider_info.json"
        save_union_raider_info_to_json(union_raider_data, filename)
        print(f"랭커들의 유니온 레이더 정보가 {filename}에 저장되었습니다.")
    else:
        print("유니온 레이더 정보를 가져오는데 실패했습니다.")

if __name__ == "__main__":
    asyncio.run(main())

랭커들의 스탯 정보가 maplestory_stat_info.json에 저장되었습니다.


In [ ]:
import asyncio
import aiohttp
import json
import os
from datetime import datetime, timedelta

import nest_asyncio
nest_asyncio.apply()

BASE_URL = "https://open.api.nexon.com/maplestory/v1"
NEXON_API = os.getenv("NEXON_API_KEY")

async def get_api_data(session, endpoint, params=None):
    headers = {"x-nxopen-api-key": NEXON_API}
    url = f"{BASE_URL}{endpoint}"
    retry_attempts = 5  # 최대 재시도 횟수
    for attempt in range(retry_attempts):
        try:
            async with session.get(url, headers=headers, params=params) as response:
                if response.status == 200:
                    return await response.json()
                else:
                    error_text = await response.text()
                    print(f"API 호출 실패: {response.status} - {error_text}")
                    return None
        except Exception as e:
            print(f"API 호출 중 오류 발생: {e}")
            return None
    print("재시도 횟수를 초과했습니다.")
    return None
    

async def get_character_stats(session, character_name):
    id_data = await get_api_data(session, "/id", {"character_name": character_name})
    if not id_data or 'ocid' not in id_data:
        print(f"랭커의 OCID를 가져오는데 실패했습니다.")
        return None

    ocid = id_data['ocid']
    yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
    params = {"ocid": ocid, "date": yesterday}
    
    union_raider_info = await get_api_data(session, "/character/stat", params)

    if union_raider_info:
        # 날짜 정보 제거
        union_raider_info.pop('date', None)
        return {character_name: union_raider_info}  # 캐릭터 이름으로 저장
    else:
        print(f"랭커의 유니온 레이더 정보를 가져오는데 실패했습니다.")
        return None

async def get_multiple_characters_info(character_names):
    async with aiohttp.ClientSession() as session:
        tasks = [get_character_stats(session, name) for name in character_names]
        results = await asyncio.gather(*tasks)
        return {k: v for d in results if d for k, v in d.items()}  # Flatten the list of dictionaries into a single dictionary

def save_character_stat_info_to_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

async def main():
    character_names = [
        "아델", "아크", "시프", "도적", "맑음",
        '건물주', '페이커', '오지환', '휴씨엔버',
        '맑아진하루', '유진', '토르', '격류'
    ]  # 원하는 캐릭터 이름 리스트

    character_info_data = await get_multiple_characters_info(character_names)
    
    if character_info_data:
        filename = "maplestory_stat_info.json"
        save_character_stat_info_to_json(character_info_data, filename)
        print(f"랭커들의 스탯 정보가 {filename}에 저장되었습니다.")
    else:
        print("랭커들의 스탯 정보를 가져오는데 실패했습니다.")

if __name__ == "__main__":
    asyncio.run(main())

API 호출 실패: 429. 재시도 중... 대기 시간: 1초
API 호출 실패: 429. 재시도 중... 대기 시간: 1초
API 호출 실패: 429. 재시도 중... 대기 시간: 1초
API 호출 실패: 429. 재시도 중... 대기 시간: 1초
API 호출 실패: 429. 재시도 중... 대기 시간: 1초
API 호출 실패: 429. 재시도 중... 대기 시간: 1초
API 호출 실패: 429. 재시도 중... 대기 시간: 1초
API 호출 실패: 429. 재시도 중... 대기 시간: 1초
API 호출 실패: 429. 재시도 중... 대기 시간: 1초
API 호출 실패: 429. 재시도 중... 대기 시간: 1초
API 호출 실패: 429. 재시도 중... 대기 시간: 1초
API 호출 실패: 429. 재시도 중... 대기 시간: 2초
API 호출 실패: 429. 재시도 중... 대기 시간: 2초
API 호출 실패: 429. 재시도 중... 대기 시간: 2초
API 호출 실패: 429. 재시도 중... 대기 시간: 2초
API 호출 실패: 429. 재시도 중... 대기 시간: 2초
API 호출 실패: 429. 재시도 중... 대기 시간: 2초
API 호출 실패: 429. 재시도 중... 대기 시간: 2초
API 호출 실패: 429. 재시도 중... 대기 시간: 2초
API 호출 실패: 429. 재시도 중... 대기 시간: 2초
API 호출 실패: 429. 재시도 중... 대기 시간: 2초
API 호출 실패: 429. 재시도 중... 대기 시간: 2초
API 호출 실패: 429. 재시도 중... 대기 시간: 4초
API 호출 실패: 429. 재시도 중... 대기 시간: 4초
API 호출 실패: 429. 재시도 중... 대기 시간: 4초
API 호출 실패: 429. 재시도 중... 대기 시간: 4초
API 호출 실패: 429. 재시도 중... 대기 시간: 4초
API 호출 실패: 429. 재시도 중... 대기 시간: 4초
API 호출 실패: 429. 재시도 

In [1]:
def calculate_battle_power(character_stats):
    # 캐릭터 스탯에서 필요한 값 추출
    base_attack = float(character_stats["공격력"])
    damage_percent = float(character_stats["데미지"])
    boss_damage_percent = float(character_stats["보스 몬스터 데미지"])
    critical_damage_percent = float(character_stats["크리티컬 데미지"])
    critical_rate = float(character_stats["크리티컬 확률"])
    final_damage_percent = float(character_stats["최종 데미지"])
    
    # 주스탯 및 부스탯
    str_stat = float(character_stats.get("STR", 0))
    dex_stat = float(character_stats.get("DEX", 0))
    int_stat = float(character_stats.get("INT", 0))
    luk_stat = float(character_stats.get("LUK", 0))
    
    # 주스탯 합산
    total_stat = str_stat + dex_stat + int_stat + luk_stat
    
    # 두 번째로 높은 스탯 계산
    stats = [str_stat, dex_stat, int_stat, luk_stat]
    stats.sort(reverse=True)  # 내림차순 정렬
    second_highest_stat = stats[1] if len(stats) > 1 else 0  # 두 번째로 높은 스탯

    # 전투력 계산식
    battle_power = (
        base_attack *
        (1 + damage_percent / 100) *
        (1 + boss_damage_percent / 100) *
        (1 + (critical_rate / 100) * (critical_damage_percent / 100)) *
        (1 + final_damage_percent / 100) *
        (1 + total_stat / 100) *  # 주스탯 적용
        (1 + second_highest_stat / 100)  # 부스탯 적용
    )

    return battle_power

# 예시 캐릭터 스탯
character_example = {
    "공격력": 11396,
    "데미지": 111.00,
    "보스 몬스터 데미지": 597.00,
    "크리티컬 확률": 90,
    "크리티컬 데미지": 104.7,
    "최종 데미지": 88.76,
    "STR": 11702,
    "DEX": 15856,
    "INT": 11122,
    "LUK": 112162
}

# 전투력 계산
battle_power = calculate_battle_power(character_example)
print(f"전투력: {battle_power:.2f}")

전투력: 147988538286.67


In [19]:
import json
import numpy as np
import faiss
from openai import OpenAI
import os

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# JSON 파일에서 유니온 공격대 데이터 로드
def load_union_raider_data(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        return json.load(file)

# 임베딩 생성 함수
def get_embedding(text):
    response = client.embeddings.create(input=[text], model="text-embedding-ada-002")
    return response.data[0].embedding

# 유니온 공격대 데이터를 텍스트로 변환
def create_union_raider_texts(union_data):
    raider_texts = []
    raider_metadata = []
    for i, raider in enumerate(union_data['rankers']):
        inner_stats = ', '.join(f"{stat['stat_field_id']}: {stat['stat_field_effect']}" for stat in raider['union_inner_stat'])
        text = (
            f"랭커 {i+1}: "
            f"유니온 공격대 스탯: {', '.join(raider['union_raider_stat'])}, "
            f"점령 스탯: {', '.join(raider['union_occupied_stat'])}, "
            f"내부 스탯: {inner_stats}"
        )
        raider_texts.append(text)
        raider_metadata.append({
            "ranker_id": i+1,
            "union_raider_stat": raider['union_raider_stat'],
            "union_occupied_stat": raider['union_occupied_stat'],
            "union_inner_stat": raider['union_inner_stat'],
            "union_block": raider['union_block']
        })
    return raider_texts, raider_metadata

# 메인 함수
def main():
    # 유니온 공격대 데이터 로드
    union_data = load_union_raider_data('maplestory_union_raider_info.json')

    # 유니온 공격대 텍스트 및 메타데이터 생성
    raider_texts, raider_metadata = create_union_raider_texts(union_data)

    # 임베딩 생성
    print("임베딩 생성 중...")
    embeddings = [get_embedding(text) for text in raider_texts]

    # FAISS 인덱스 생성
    print("FAISS 인덱스 생성 중...")
    dimension = len(embeddings[0])
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings).astype('float32'))

    # 인덱스와 메타데이터 저장
    print("인덱스 및 메타데이터 저장 중...")
    faiss.write_index(index, "maplestory_union_raider_index.faiss")
    with open("union_raider_metadata.json", 'w', encoding='utf-8') as f:
        json.dump(raider_metadata, f, ensure_ascii=False, indent=2)

    print("FAISS 인덱스가 성공적으로 생성되고 저장되었습니다.")

    # 검색 함수
    def search_union_raider(query, k=5):
        query_embedding = get_embedding(query)
        D, I = index.search(np.array([query_embedding]).astype('float32'), k)
        return [(raider_texts[i], raider_metadata[i], D[0][j]) for j, i in enumerate(I[0])]

    # 검색 예제 실행
    print("검색 예제 실행 중...")
    results = search_union_raider("높은 보스 데미지와 크리티컬 확률")
    for text, metadata, distance in results:
        print(f"결과: {text}")
        print(f"메타데이터: {metadata}")
        print(f"거리: {distance}")
        print()

if __name__ == "__main__":
    main()

임베딩 생성 중...
FAISS 인덱스 생성 중...
인덱스 및 메타데이터 저장 중...
FAISS 인덱스가 성공적으로 생성되고 저장되었습니다.
검색 예제 실행 중...
결과: 랭커 2: 유니온 공격대 스탯: 보스 몬스터 공격 시 데미지 6% 증가, 공격력/마력 20 증가, LUK 80 증가, STR 80 증가, INT 80 증가, 스킬 재사용 대기시간 6% 감소, DEX 80 증가, 공격 시 20%의 확률로 데미지 20% 증가, STR 80 증가, STR 80 증가, STR 100 증가, INT 100 증가, 크리티컬 확률 4% 증가, 크리티컬 데미지 6% 증가, LUK 100 증가, 크리티컬 확률 5% 증가, 방어율 무시 5% 증가, STR 80 증가, 버프 지속시간 25% 증가, STR 80 증가, STR, DEX, LUK 50 증가, 경험치 획득량 10% 증가, DEX 80 증가, STR 80 증가, STR 80 증가, DEX 80 증가, DEX 100 증가, 상태 이상 내성 4 증가, INT 80 증가, 최대 HP 2000 증가, INT 80 증가, INT 80 증가, LUK 80 증가, 최대 MP 5% 증가, LUK 80 증가, LUK 80 증가, LUK 80 증가, 메소 획득량 4% 증가, INT 80 증가, 점령 스탯: STR 75 증가, 크리티컬 데미지 20.00% 증가, DEX 5 증가, 상태 이상 내성 1 증가, 보스 몬스터 공격 시 데미지 40% 증가, 공격력 15 증가, 버프 지속시간 20% 증가, 방어율 무시 40% 증가, 내부 스탯: 0: 유니온 LUK, 1: 유니온 마력, 2: 유니온 최대 MP, 3: 유니온 공격력, 4: 유니온 DEX, 5: 유니온 STR, 6: 유니온 INT, 7: 유니온 최대 HP
메타데이터: {'ranker_id': 2, 'union_raider_stat': ['보스 몬스터 공격 시 데미지 6% 증가', '공격력/마력 20 증가', 'LUK 80 증가', 'STR 80 증가', 'INT 80 증가', '스킬

In [66]:
import json
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

def merge_json_files(directory):
    merged_data = {}
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                data = json.load(file)
                merged_data.update(data)
    return merged_data

# 직업 클래스별 JSON 파일이 있는 디렉토리
directory = './job_class'
merged_data = merge_json_files(directory)

# 통합된 데이터를 새 JSON 파일로 저장
with open('merged_jobs.json', 'w', encoding='utf-8') as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=2)


In [69]:
import json
import numpy as np
import faiss
from openai import OpenAI
import os

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# JSON 파일에서 스킬 데이터 로드
def load_skill_data(directory):
    merged_data = {}
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                data = json.load(file)
                merged_data.update(data)
    return merged_data

# 임베딩 생성 함수
def get_embedding(text):
    response = client.embeddings.create(input=[text], model="text-embedding-ada-002")
    return response.data[0].embedding

# 스킬 데이터를 텍스트로 변환
def create_skill_texts(skill_data):
    skill_texts = []
    skill_metadata = []
    for job_class, skills in skill_data.items():
        for skill in skills['character_skill']:
            text = (
                f"직업: {job_class}, 스킬 이름: {skill['skill_name']}, "
                f"설명: {skill['skill_description']}, "
                f"레벨: {skill['skill_level']}, 효과: {skill['skill_effect']}"
            )
            skill_texts.append(text)
            skill_metadata.append({
                "job_class": job_class,
                "skill_name": skill['skill_name'],
                "skill_level": skill['skill_level'],
                "skill_effect": skill['skill_effect']
            })
    return skill_texts, skill_metadata

# 메인 함수
def main():
    # 스킬 데이터 로드
    skill_data = load_skill_data('./job_class')

    # 스킬 텍스트 및 메타데이터 생성
    skill_texts, skill_metadata = create_skill_texts(skill_data)

    # 임베딩 생성
    print("임베딩 생성 중...")
    embeddings = [get_embedding(text) for text in skill_texts]

    # FAISS 인덱스 생성
    print("FAISS 인덱스 생성 중...")
    dimension = len(embeddings[0])
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings).astype('float32'))

    # 인덱스와 메타데이터 저장
    print("인덱스 및 메타데이터 저장 중...")
    faiss.write_index(index, "maplestory_skill_index.faiss")
    with open("skill_metadata.json", 'w', encoding='utf-8') as f:
        json.dump(skill_metadata, f, ensure_ascii=False, indent=2)

    print("FAISS 인덱스가 성공적으로 생성되고 저장되었습니다.")

    # 검색 함수
    def search_skill(query, k=5):
        query_embedding = get_embedding(query)
        D, I = index.search(np.array([query_embedding]).astype('float32'), k)
        return [(skill_texts[i], skill_metadata[i], D[0][j]) for j, i in enumerate(I[0])]

    # 검색 예제 실행
    print("검색 예제 실행 중...")
    results = search_skill("강력한 공격 스킬")
    for text, metadata, distance in results:
        print(f"결과: {text}")
        print(f"메타데이터: {metadata}")
        print(f"거리: {distance}")
        print()

if __name__ == "__main__":
    main()

임베딩 생성 중...
FAISS 인덱스 생성 중...
인덱스 및 메타데이터 저장 중...
FAISS 인덱스가 성공적으로 생성되고 저장되었습니다.
검색 예제 실행 중...
결과: 직업: 제로, 스킬 이름: 기가 크래시, 설명: [마스터 레벨 : 10]
검을 지면으로 강하게 내려치며 전방에 있는 적을 공격한다., 레벨: 10, 효과: 타임 포스 15 소비, 최대 6명의 적을 250%의 데미지로 6번 공격
재사용 대기시간 15초
메타데이터: {'job_class': '제로', 'skill_name': '기가 크래시', 'skill_level': 10, 'skill_effect': '타임 포스 15 소비, 최대 6명의 적을 250%의 데미지로 6번 공격\n재사용 대기시간 15초'}
거리: 0.23975875973701477

결과: 직업: 카이저, 스킬 이름: 소드 스트라이크, 설명: [마스터 레벨 : 20]
여러 개의 검을 소환하여 내려꽂으며 폭발시켜 적들을 공격한다.
커맨드 스킬 : 공격 도중 + ↓커맨드, 레벨: 20, 효과: MP 45 소비, 최대 12명의 적을 480% 데미지로 5번 공격
메타데이터: {'job_class': '카이저', 'skill_name': '소드 스트라이크', 'skill_level': 20, 'skill_effect': 'MP 45 소비, 최대 12명의 적을 480% 데미지로 5번 공격'}
거리: 0.24653086066246033

결과: 직업: 카이저, 스킬 이름: 드래곤슬래시 3차 강화, 설명: [마스터 레벨 : 6]
드래곤슬래시의 공격력을 강화한다.필요 스킬 : 드래곤슬래시 2차 강화 2레벨 이상, 레벨: 6, 효과: 드래곤슬래시의 데미지 90% 추가 강화
메타데이터: {'job_class': '카이저', 'skill_name': '드래곤슬래시 3차 강화', 'skill_level': 6, 'skill_effect': '드래곤슬래시의 데미지 90% 추가 강화'}
거리: 0.24815160036087036

결과: 직업: 제논,

In [2]:
import json
import numpy as np
import faiss
from openai import OpenAI
import os

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# JSON 파일에서 데이터 로드
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# 임베딩 생성 함수
def get_embedding(text):
    response = client.embeddings.create(input=[text], model="text-embedding-ada-002")
    return response.data[0].embedding  # 올바른 속성 접근

# 메인 함수
def main():
    # 골드 주화 계산 데이터 로드
    gold_coin_data = load_data('azumosu.json')

    # 데이터를 담을 리스트
    skill_texts = []
    skill_metadata = []

    # JSON 데이터에서 텍스트 및 메타데이터 생성
    for example in gold_coin_data['currency_analysis_examples']:
        text = (
            f"골드 주화 가격: {example['gold_coin_price']}, "
            f"메이플포인트 가격: {example['meso_market_rate']}, "
            f"손익분기점: {example['total_calculation']['break_even_point']}, "
            f"이득 여부: {example['total_calculation']['maple_point_profit_difference']} 메이플포인트"
        )
        skill_texts.append(text)
        skill_metadata.append(example)

    # 임베딩 생성
    print("임베딩 생성 중...")
    embeddings = [get_embedding(text) for text in skill_texts]

    # FAISS 인덱스 생성
    print("FAISS 인덱스 생성 중...")
    dimension = len(embeddings[0])
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings).astype('float32'))

    # 인덱스와 메타데이터 저장
    print("인덱스 및 메타데이터 저장 중...")
    faiss.write_index(index, "gold_coin_index.faiss")
    with open("gold_coin_metadata.json", 'w', encoding='utf-8') as f:
        json.dump(skill_metadata, f, ensure_ascii=False, indent=2)

    print("FAISS 인덱스가 성공적으로 생성되고 저장되었습니다.")

    # 검색 함수
    def search_gold_coin(query, k=5):
        query_embedding = get_embedding(query)
        D, I = index.search(np.array([query_embedding]).astype('float32'), k)
        return [(skill_texts[i], skill_metadata[i], D[0][j]) for j, i in enumerate(I[0])]

    # 검색 예제 실행
    print("검색 예제 실행 중...")
    results = search_gold_coin("이득이 되는 골드 주화 가격은?")
    for text, metadata, distance in results:
        print(f"결과: {text}")
        print(f"메타데이터: {metadata}")
        print(f"거리: {distance}")
        print()

if __name__ == "__main__":
    main()


임베딩 생성 중...
FAISS 인덱스 생성 중...
인덱스 및 메타데이터 저장 중...
FAISS 인덱스가 성공적으로 생성되고 저장되었습니다.
검색 예제 실행 중...
결과: 골드 주화 가격: 48000000, 메이플포인트 가격: 2300, 손익분기점: 2191, 이득 여부: 109 메이플포인트
메타데이터: {'gold_coin_price': 48000000, 'meso_market_rate': 2300, 'gold_coin_count': 3, 'azmos_coin_count': 25, 'required_maple_points': 2500, 'gold_coin_conversion_fee_per_coin': 300, 'total_calculation': {'gold_coin_total_value': 144000000, 'total_maple_points_used': 3400, 'break_even_point': 2191, 'maple_point_profit_difference': 109, 'meso_profit_difference': 7910000}}
거리: 0.24739301204681396

결과: 골드 주화 가격: 51800000, 메이플포인트 가격: 1900, 손익분기점: 2191, 이득 여부: -291 메이플포인트
메타데이터: {'gold_coin_price': 51800000, 'meso_market_rate': 1900, 'gold_coin_count': 3, 'azmos_coin_count': 25, 'required_maple_points': 2500, 'gold_coin_conversion_fee_per_coin': 300, 'total_calculation': {'gold_coin_total_value': 155400000, 'total_maple_points_used': 3400, 'break_even_point': 2191, 'maple_point_profit_difference': -291, 'meso_profit_difference'

In [7]:
import json
import numpy as np
import faiss
from openai import OpenAI
import os

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# JSON 파일에서 데이터 로드
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# 임베딩 생성 함수
def get_embedding(text):
    response = client.embeddings.create(input=[text], model="text-embedding-ada-002")
    return response.data[0].embedding

# 메인 함수
def main():
    # 메이플스토리 경제 시스템 데이터 로드
    maplestory_data = load_data('maple_economy.json')

    # 데이터를 담을 리스트
    texts = []
    metadata = []

    # JSON 데이터에서 텍스트 및 메타데이터 생성
    for key, value in maplestory_data.items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                text = f"{key} - {sub_key}: {sub_value}"
                texts.append(text)
                metadata.append({"category": key, "subcategory": sub_key})
        elif isinstance(value, list):
            for item in value:
                text = f"{key}: {item}"
                texts.append(text)
                metadata.append({"category": key})
        else:
            text = f"{key}: {value}"
            texts.append(text)
            metadata.append({"category": key})

    # 임베딩 생성
    print("임베딩 생성 중...")
    embeddings = [get_embedding(text) for text in texts]

    # FAISS 인덱스 생성
    print("FAISS 인덱스 생성 중...")
    dimension = len(embeddings[0])
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings).astype('float32'))

    # 인덱스와 메타데이터 저장
    print("인덱스 및 메타데이터 저장 중...")
    faiss.write_index(index, "maplestory_economy_index.faiss")
    with open("maplestory_economy_metadata.json", 'w', encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=2)

    print("FAISS 인덱스가 성공적으로 생성되고 저장되었습니다.")

    # 검색 함수
    def search_maplestory_economy(query, k=5):
        query_embedding = get_embedding(query)
        D, I = index.search(np.array([query_embedding]).astype('float32'), k)
        return [(texts[i], metadata[i], D[0][j]) for j, i in enumerate(I[0])]

    # 검색 예제 실행
    print("검색 예제 실행 중...")
    results = search_maplestory_economy("메이플스토리의 주요 재화는 무엇인가요?")
    for text, meta, distance in results:
        print(f"결과: {text}")
        print(f"메타데이터: {meta}")
        print(f"거리: {distance}")
        print()

if __name__ == "__main__":
    main()

임베딩 생성 중...
FAISS 인덱스 생성 중...
인덱스 및 메타데이터 저장 중...
FAISS 인덱스가 성공적으로 생성되고 저장되었습니다.
검색 예제 실행 중...
결과: 메이플스토리_경제시스템 - 재화: [{'이름': '메소', '설명': '게임 내 기본 화폐로, 몬스터 사냥이나 보스 몬스터 처치로 획득 가능', '특징': '이론상 무한히 생산 가능한 재화'}, {'이름': '메이플포인트', '설명': '현금과 1:1 가치를 가진 재화, 메소마켓에서 메소와 교환 가능', '특징': '수요와 공급에 따라 시세가 결정되며, 캐시 아이템 구매에 사용'}, {'이름': '메이플 주화', '설명': '에픽 던전 완료나 이벤트를 통해 획득 가능한 새로운 재화', '특징': ['NPC를 통해 메소로 교환 가능', '교환 시 300 메이플포인트의 수수료 필요', '1 주화 = 5천만 메소 (수수료 별도)', '가치는 게임 내 메소 저장량, 생산량, 메이플포인트 시세에 따라 변동']}]
메타데이터: {'category': '메이플스토리_경제시스템', 'subcategory': '재화'}
거리: 0.28940773010253906

결과: 메이플스토리_경제시스템 - 경제_특징: ['게임 내 경제의 균형 유지', '현금과 게임 내 재화 간의 연결 제공', '인플레이션 조절', '메이플 주화 도입으로 메소의 가치 안정화 및 불법 현금거래 억제']
메타데이터: {'category': '메이플스토리_경제시스템', 'subcategory': '경제_특징'}
거리: 0.3433310389518738

결과: 메이플스토리N_경제시스템 - 기대_효과: ['마켓플레이스 시세 안정', '유저에게 유연한 선택 유도', '다양한 아이템 빌드와 폭넓은 지역 탐험 촉진']
메타데이터: {'category': '메이플스토리N_경제시스템', 'subcategory': '기대_효과'}
거리: 0.3450925350189209

결과: 메이플스토리N_경제시스템 - 목표: ['실물 경제와 유사한 게임 경